# Project 1


## Description

## Data import

In [49]:
import dlc_practical_prologue as prologue

# We import 1000 pairs of digits for the training and the testing inputs, targets and classes
number_of_pairs = 1000 
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets( 
    number_of_pairs)

# Quick verification of the sizes:
print('Size of train_input:', train_input.size())
print('Size of train_target:', test_target.size())
print('Size of train_classes:', test_classes.size())
print('Size of test_input:', test_input.size())
print('Size of test_target:', test_target.size())
print('Size of test_classes:', test_classes.size())



Size of train_input: torch.Size([1000, 2, 14, 14])
Size of train_target: torch.Size([1000])
Size of train_classes: torch.Size([1000, 2])
Size of test_input: torch.Size([1000, 2, 14, 14])
Size of test_target: torch.Size([1000])
Size of test_classes: torch.Size([1000, 2])


## CNN

In [50]:
import torch
from torch import nn
from torch.nn import functional as F

class ConvNet(nn.Module):
    def __init__(self, nb_hidden=200):
        super().__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

In [51]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25, printAccLoss = False):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-1

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()

            model.zero_grad()
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad

        if printAccLoss:
            print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k] != predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors

In [52]:
mini_batch_size = 50

for k in range(10):
    model = ConvNet()
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0), nb_test_errors, test_input.size(0)))

test error Net 44.80% 448/1000
test error Net 55.20% 552/1000
test error Net 44.80% 448/1000
test error Net 44.80% 448/1000
test error Net 44.80% 448/1000
test error Net 55.20% 552/1000
test error Net 44.80% 448/1000
test error Net 44.80% 448/1000
test error Net 44.80% 448/1000
test error Net 44.80% 448/1000
